In [1]:

import ee
import pandas as pd
import geemap
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:

# Authenticate and initialize Earth Engine
#ee.Authenticate()
ee.Initialize()

In [3]:
# Date range
start_date = '2016-06-01'
end_date = '2025-06-14'

In [ ]:
# def file_read(csv_path):
#     csv_path = csv_path # Update this if needed
#     df = pd.read_csv(csv_path)
    
#     # Ensure column names
#     lat_col, lon_col = 'latitude', 'longitude'  # Modify if your columns are named differently

#     # Extract bounding box
#     min_lat = df[lat_col].min()
#     max_lat = df[lat_col].max()
#     min_lon = df[lon_col].min()
#     max_lon = df[lon_col].max()

#     # Create EE geometry bounding box
#     region_bbox = ee.Geometry.BBox(min_lon, min_lat, max_lon, max_lat)
#     print(f"✅ Bounding box created: ({min_lon}, {min_lat}) to ({max_lon}, {max_lat})")
#     features = []
#     for _, row in df.iterrows():
#         point = ee.Geometry.Point(row[lon_col], row[lat_col])
#         feature = ee.Feature(point, {
#             'Latitude': row[lat_col],
#             'Longitude': row[lon_col]
#         })
#         features.append(feature)
#     point_fc = ee.FeatureCollection(features)
#     return df,region_bbox,point_fc

# # Sentinel-2 NDVI calculation function
# def compute_ndvi(image):
#     ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
#     return image.addBands(ndvi)

# # Define cloud masking function using QA60 band (clouds and cirrus)
# def mask_s2_clouds(image):
#     qa = image.select('QA60')
#     # Bits 10 and 11 are clouds and cirrus
#     cloud_bit_mask = 1 << 10
#     cirrus_bit_mask = 1 << 11
#     mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
#            qa.bitwiseAnd(cirrus_bit_mask).eq(0))
#     return image.updateMask(mask).copyProperties(image, ['system:time_start'])

# def s2_collection(start_date, end_date, region_bbox):
#     # Load Sentinel-2 surface reflectance, apply filters
#     s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
#         .filterDate(start_date, end_date) \
#         .filterBounds(region_bbox) \
#         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
#         .map(mask_s2_clouds) \
#         .map(compute_ndvi) \
#         .select('NDVI')
#     # Print metadata: number of images
#     # count = s2_col.size().getInfo()
#     # print(f"✅ Total images in filtered collection: {count}")
#     return s2_col

# def display_image(im,region_bbox, point_fc):
#     first_info = im.getInfo()
#     print("\n📄 Metadata of first image in collection:")
#     print(f"ID: {first_info['id']}")
#     print(f"Date: {ee.Date(im.get('system:time_start')).format('YYYY-MM-dd').getInfo()}")

#     # Visualize NDVI of first image
#     Map = geemap.Map(center=[10, 0], zoom=4)  # Adjust center for your region
#     ndvi_vis = {'min': 0, 'max': 1, 'palette': ['blue', 'white', 'green']}
#     Map.addLayer(im.select('NDVI'), ndvi_vis, 'First NDVI Image')
#     Map.addLayer(region_bbox,{},'BBOX')
#     Map.addLayer(point_fc, {'color': 'red'}, 'Stations')
#     Map.add_colorbar(vis_params=ndvi_vis, label='NDVI', layer_name='First NDVI Image')
#     return Map


# # Function to extract NDVI time series for a point
# def extract_ndvi_series(lat, lon,s2_col):
#     point = ee.Geometry.Point(float(lon), float(lat))
#     def extract_value(image):
#         date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
#         value = image.reduceRegion(ee.Reducer.mean(), point, scale=10).get('NDVI')
#         return ee.Feature(None, {'date': date, 'NDVI': value})
#     features = s2_col.map(extract_value).filter(ee.Filter.notNull(['NDVI']))
#     return features.aggregate_array('date').getInfo(), features.aggregate_array('NDVI').getInfo()


In [ ]:
# def batch_extract(df,s2_col):
#     # Prepare output dataframe
#     results = []

#     for idx, row in tqdm(df.iterrows(), total=len(df)):
#         lat, lon, network,station = row['latitude'], row['longitude'], row['network'], row['station']
#         try:
#             dates, values = extract_ndvi_series(lat, lon,s2_col)
#             row_data = {
#                 'network':network,
#                 'station':station,
#                 'Latitude': lat,
#                 'Longitude': lon
#             }
#             row_data.update({date: value for date, value in zip(dates, values)})
#             results.append(row_data)
#         except Exception as e:
#             print(f"Error at index {idx}: {e}")

#     # Convert result to DataFrame
#     df = pd.DataFrame(results)
#     return df


In [ ]:
# import os
# import glob
# import pandas as pd

# # Root data directory
# root_dir = 'data'
# # Output directory
# output_base = os.path.join(root_dir, 'satellite_data')

# # Get all continent folders under 'data' (e.g., africa, europe, asia)
# for continent_folder in os.listdir(root_dir):
#     continent_path = os.path.join(root_dir, continent_folder)
#     # Skip if not a directory or if it's the satellite_data output folder itself
#     if not os.path.isdir(continent_path) or continent_folder == 'satellite_data':
#         continue

#     # Path to input CSVs
#     input_csv_dir = os.path.join(continent_path, 'extracted_data', 'mean', 'agg_mean')
#     if not os.path.exists(input_csv_dir):
#         continue

#     # Output folder: data/satellite_data/<continent>
#     output_folder = os.path.join(output_base, continent_folder)
#     os.makedirs(output_folder, exist_ok=True)

#     # Process all CSV files in the input directory
#     csv_files = glob.glob(os.path.join(input_csv_dir, '*.csv'))
#     for csv_file in csv_files:
#         filename = os.path.basename(csv_file)
#         output_path = os.path.join(output_folder, filename)

#         # Skip processing if file already exists
#         if os.path.exists(output_path):
#             print(f"Skipped (already exists): {output_path}")
#             continue
        
#         df, region_bbox,point_fc = file_read(csv_file)
#         s2_col=s2_collection(start_date, end_date, region_bbox)
#         df_extract=batch_extract(df,s2_col)
#         output_path = os.path.join(output_folder, filename)
#         df_extract.to_csv(output_path, index=False)
#         print(f"Saved: {output_path}")

### updated code to avoid time out error

In [4]:
def file_read(csv_path):
    csv_path = csv_path # Update this if needed
    df = pd.read_csv(csv_path)
    
    # Ensure column names
    lat_col, lon_col = 'latitude', 'longitude'  # Modify if your columns are named differently

    # Extract bounding box
    min_lat = df[lat_col].min()
    max_lat = df[lat_col].max()
    min_lon = df[lon_col].min()
    max_lon = df[lon_col].max()

    # Create EE geometry bounding box
    region_bbox = ee.Geometry.BBox(min_lon, min_lat, max_lon, max_lat)
    print(f"✅ Bounding box created: ({min_lon}, {min_lat}) to ({max_lon}, {max_lat})")
    features = []
    for _, row in df.iterrows():
        point = ee.Geometry.Point(row[lon_col], row[lat_col])
        feature = ee.Feature(point, {
            'Latitude': row[lat_col],
            'Longitude': row[lon_col]
        })
        features.append(feature)
    point_fc = ee.FeatureCollection(features)
    return df,region_bbox,point_fc

# Sentinel-2 NDVI calculation function
def compute_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Define cloud masking function using QA60 band (clouds and cirrus)
def mask_s2_clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).copyProperties(image, ['system:time_start'])

In [5]:
def extract_ndvi_series(lat, lon, start_date, end_date, buffer_m=250):
    point = ee.Geometry.Point(float(lon), float(lat))
    buffer = point.buffer(buffer_m)

    # Query per-point collection
    s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(buffer) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(mask_s2_clouds) \
        .map(compute_ndvi) \
        .select('NDVI')

    def extract_value(image):
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        value = image.reduceRegion(ee.Reducer.mean(), point, scale=10).get('NDVI')
        return ee.Feature(None, {'date': date, 'NDVI': value})
    
    features = s2_col.map(extract_value).filter(ee.Filter.notNull(['NDVI']))
    return features.aggregate_array('date').getInfo(), features.aggregate_array('NDVI').getInfo()

def batch_extract(df, start_date, end_date):
    # Step 1: Extract time series per point and track all unique dates
    all_results = []
    all_dates_set = set()

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        lat, lon, network, station = row['latitude'], row['longitude'], row['network'], row['station']
        try:
            dates, values = extract_ndvi_series(lat, lon, start_date, end_date)
            all_dates_set.update(dates)
            row_data = {
                'network': network,
                'station': station,
                'Latitude': lat,
                'Longitude': lon,
                'ndvi_series': dict(zip(dates, values))
            }
            all_results.append(row_data)
        except Exception as e:
            print(f"❌ Error at index {idx}: {e}")

    # Step 2: Normalize to master date list
    all_dates_sorted = sorted(list(all_dates_set))  # use as fixed columns
    final_results = []

    for row in all_results:
        base_data = {
            'network': row['network'],
            'station': row['station'],
            'Latitude': row['Latitude'],
            'Longitude': row['Longitude']
        }
        # Fill in NDVI per date or NaN if missing
        ndvi_series = {date: row['ndvi_series'].get(date, float('nan')) for date in all_dates_sorted}
        base_data.update(ndvi_series)
        final_results.append(base_data)

    df_final = pd.DataFrame(final_results)
    return df_final


In [6]:
import os
import glob
import pandas as pd

# Root data directory
root_dir = 'data'
# Output directory
output_base = os.path.join(root_dir, 'satellite_data')

# Get all continent folders under 'data' (e.g., africa, europe, asia)
for continent_folder in os.listdir(root_dir):
    continent_path = os.path.join(root_dir, continent_folder)
    # Skip if not a directory or if it's the satellite_data output folder itself
    if not os.path.isdir(continent_path) or continent_folder == 'satellite_data':
        continue

    # Path to input CSVs
    input_csv_dir = os.path.join(continent_path, 'extracted_data', 'mean', 'agg_mean')
    if not os.path.exists(input_csv_dir):
        continue

    # Output folder: data/satellite_data/<continent>
    output_folder = os.path.join(output_base, continent_folder)
    os.makedirs(output_folder, exist_ok=True)

    # Process all CSV files in the input directory
    csv_files = glob.glob(os.path.join(input_csv_dir, '*.csv'))
    for csv_file in csv_files:
        filename = os.path.basename(csv_file)
        output_path = os.path.join(output_folder, filename)

        # Skip processing if file already exists
        if os.path.exists(output_path):
            print(f"Skipped (already exists): {output_path}")
            continue
        
        df, region_bbox,point_fc = file_read(csv_file)
        df_extract = batch_extract(df, start_date, end_date)
        output_path = os.path.join(output_folder, filename)
        df_extract.to_csv(output_path, index=False)
        print(f"Saved: {output_path}")

✅ Bounding box created: (1.6053, 9.74407) to (2.6956, 13.6456)


100%|██████████| 7/7 [02:45<00:00, 23.62s/it]


Saved: data\satellite_data\africa\AMMA-CATCH.csv
✅ Bounding box created: (30.4783, 13.2829) to (30.4783, 13.2829)


100%|██████████| 1/1 [00:33<00:00, 33.54s/it]


Saved: data\satellite_data\africa\SD_DEM.csv
✅ Bounding box created: (-3.93513, -2.88283) to (40.02984, 11.06886)


100%|██████████| 33/33 [06:26<00:00, 11.70s/it]

Saved: data\satellite_data\africa\TAHMO.csv
